### Import packages

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras import backend as K
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import numpy as np
from numpy.testing import assert_allclose
from keras.models import model_from_json, load_model
from keras.callbacks import ModelCheckpoint
import glob
import cv2

### Import dataset

In [ ]:
file_path = 'Dataset as Image/1.Train/**/*.png'
dataTest = []
labelTest = []  
cnumo = 0
print("-->Classifying Images. I't will take some time. Plese Wait.")
for filename in glob.iglob(file_path, recursive=True):
    FN = filename.split("1.Train")
    FN = FN[1]
    #print(FN[1])
    cnumo += 1
    image = cv2.imread(filename, 1)
    image = cv2.resize(image, (28, 28))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = image.reshape(( 28, 28,1))
    #image=np.expand_dims(image,axis=2)
    dataTest.append (image)
    lab = ord(FN[1])-65
    labelTest.append(lab)
print("-->Classification Done")
print(cnumo)
dataTest = np.array(dataTest)
labelTest = np.array(labelTest)


X = dataTest.astype("float32")/255
print(X.shape)
Y = labelTest
print("-->Converting Data")
print("Number of samples to test: ",len(dataTest))

In [ ]:
'''# seed for reproducing same results
seed = 785
np.random.seed(seed)

# load dataset("Image is stored in CSV file to save image to array conversion time")
print("Loading Image File. This will take some time. Please be patient")
dataset = np.loadtxt('Dataset as csv/A_Z Handwritten Data.csv', delimiter=',')'''
print("Image dataset loaded")

### Preprocessing

In [ ]:
# split into input and output variables
#X = dataset[:,0:784]
#Y = dataset[:,0]

# seed for reproducing same results
seed = 785
np.random.seed(seed)

# split the data into training (50%) and testing (50%)
(X_train, X_val, Y_train, Y_val) = train_test_split(X, Y, test_size=0.30, random_state=seed)

#convert image resolution to 28x28 image
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')
X_val = X_val.reshape(X_val.shape[0], 28, 28, 1).astype('float32')

X_train = X_train / 255
X_val = X_val / 255

# one hot encode outputs
Y_train = np_utils.to_categorical(Y_train)
Y_val = np_utils.to_categorical(Y_val)

num_classes = Y_val.shape[1]
print("Total Classes:",num_classes)

### Build Model

In [ ]:
# create model
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(28, 28, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

### Compile Model

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### Training Phase

In [ ]:
# define the checkpoint
filepath = "model-v3.h5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

# fit the model
model.fit(X_train, Y_train, epochs=100, batch_size=200, callbacks=callbacks_list, validation_data=(X_val, Y_val))

### Load trained file and resume training

In [ ]:
#Run this module if you need to <<Resume Training>>. This is not an incremental training. 
#This is like resume training with the old data set.

#Compile model
new_model = load_model(filepath)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit the model
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
new_model.fit(X_train, Y_train, epochs=0, batch_size=200, callbacks=callbacks_list, validation_data=(X_val, Y_val))
model.save(filepath)

In [ ]:
#Returns the loss value & metrics values for the model in test mode.
new_model = load_model(filepath)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
scoreEval = new_model.evaluate(X_val, Y_val, verbose=1)
print("CNN Error: %.2f%%" % (100-scoreEval[1]*100))
print("Done")

##### Shutdown the kernel after training. 
    Shutdown the kernel to release the used up memory.